# LLM as judge

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import TextStreamer
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", attn_implementation="flash_attention_2", trust_remote_code=True, torch_dtype=torch.bfloat16)

[2024-09-14 22:27:14,128] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Process ForkProcess-5:
Process ForkProcess-32:
Process ForkProcess-13:
Process ForkProcess-25:
Process ForkProcess-3:
Process ForkProcess-12:
Process ForkProcess-6:
Process ForkProcess-17:
Process ForkProcess-21:
Process ForkProcess-31:
Process ForkProcess-29:
Process ForkProcess-30:
Process ForkProcess-24:
Process ForkProcess-4:
Process ForkProcess-22:
Process ForkProcess-18:
Process ForkProcess-20:
Process ForkProcess-14:
Process ForkProcess-23:
Process ForkProcess-11:
Process ForkProcess-28:
Process ForkProcess-1:
Process ForkProcess-19:
Process ForkProcess-15:
Process ForkProcess-27:
Process ForkProcess-26:
Process ForkProcess-16:
Process ForkProcess-2:
Process ForkProcess-10:
Process ForkProcess-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceba

In [1]:
#v2
PROMPT = """Determine whether the provided claim is grounded in the corresponding document.

In this context, grounding means that the information presented in the claim is substantiated by the document. If not, the claim should be considered not grounded.

Document: {document}

Claim: {claim}

Please assess whether the claim is grounded in the document by responding with either ’yes’ or ’no.’

Answer:"""

In [ ]:
def get_mixtral_scores(data_i):
    import numpy as np
    output = []
    for i in data_i['mixtral_scores']:
        output.append('yes' in i.lower())
    return np.mean(output)

In [ ]:
def get_llm_as_judge_score(data_i):
    # get question_related_knowledge segments
    gold_ctxs = []
    for j in i['ctxs']:
        if j.get('label',False) or j.get('isgold',False):
            gold_ctxs.append(j)
    passages = [g['title']+' '+g['text'] for g in gold_ctxs]
    question = i['question']
    llm_as_judge_score = []
    for passage in passages:
        prompt = PROMPT.format(document=passage, claim=predict)
        inputs = tokenizer(prompt, return_tensors='pt', add_special_tokens=False)
        length = inputs['input_ids'].size(1)
        outputs = model.generate(**inputs, max_new_tokens=64, streamer=streamer, pad_token_id=tokenizer.eos_token_id)
        llm_as_judge_i = tokenizer.decode(outputs[0][length:], skip_special_tokens=True)
        llm_as_judge_score.append(llm_as_judge_i)
    output = ['yes' in i.lower() for i in llm_as_judge_score]
    return np.mean(output)